# 02 RAG | 02 Embeddings | 02 Image Embeddings

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.

The capability to generate vectors from images and text within the same multi-dimensional space is provided by Azure AI Vision. The current Azure AI Vision .NET SDK does not support this functionality, so REST API calls are used to interact with the Azure AI Vision instance to achieve this integration.

## Step 1: Load environment variables

In [1]:
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: Azure.Storage.Blobs, 12.19.0"

using System.IO;
using DotNetEnv;

string configurationFile = "../../Configuration/application.env";
Env.Load(configurationFile);

string visionApiKey = Env.GetString("WS_VISION_APIKEY");
string visionEndPoint = Env.GetString("WS_VISION_ENDPOINT");
string storageConnectionString = Env.GetString("WS_STORAGE_CONNECTIONSTRING");

Console.WriteLine("Nuget packages installed...");
Console.WriteLine("Configuration file loaded...");


Installed Packages Azure.Storage.Blobs, 12.19.0 DotNetEnv, 2.5.0

Nuget packages installed...
Configuration file loaded...


## Step 2: Load helper class

The [helper class](02_ImageEmbeddingsHelper.csx) provides functionality to: 

- `GetTextEmbedding()` using the deployed Azure AI Vision instance
- `GetImageEmbedding()` using the deployed Azure AI Vision instance
- `UploadLocalFile()` to upload an image file to an Azure blob storage 

In [2]:
#!import "02_ImageEmbeddingsHelper.csx"

Console.WriteLine($"Class AiVisionHelper with methods:");
Console.WriteLine($"- GetTextEmbedding(string text)");
Console.WriteLine($"- GetImageEmbedding(string imageUrl)");
Console.WriteLine($"- UploadLocalFile(string localFilePath, string containerName, string blobName)");
Console.WriteLine($"created...");

Class AiVisionHelper with methods:
- GetTextEmbedding(string text)
- GetImageEmbedding(string imageUrl)
- UploadLocalFile(string localFilePath, string containerName, string blobName)
created...


## Step 2: Create image embedding

Three embeddings from an image provided in the assets folder showing an US american team winning the Super Bowl in different styles are created.


In [3]:
AiVisionHelper _aiVisionHelper = new AiVisionHelper(visionApiKey, visionEndPoint, storageConnectionString);

// Embeding from photo realistic image
string fileName = "../../Assets/Image/SuperBowl_Photorealistic.png";
Uri sasUri = await _aiVisionHelper.UploadLocalFile(fileName, "architecture", Path.GetFileName(fileName));
float[] imageEmbeddingPhotorealistic = await _aiVisionHelper.GetImageEmbedding(sasUri.ToString());

// Embeding from sketch style image
fileName = "../../Assets/Image/SuperBowl_Photorealistic.png";
sasUri = await _aiVisionHelper.UploadLocalFile(fileName, "architecture", Path.GetFileName(fileName));
float[] imageEmbeddingSketch = await _aiVisionHelper.GetImageEmbedding(sasUri.ToString());

// Embeding from oilpainting style
fileName = "../../Assets/Image/SuperBowl_Oilpainting.png";
sasUri = await _aiVisionHelper.UploadLocalFile(fileName, "architecture", Path.GetFileName(fileName));
float[] imageEmbeddingOilpainting = await _aiVisionHelper.GetImageEmbedding(sasUri.ToString());

Console.WriteLine("Image embeddings created...");

Image embeddings created...


## Step 3: Create text embedding

An embedding from the text literal *US American Team winning the Super Bowl" will be created using the same model in the above code cell to create an embedding out of an image.

In [4]:
string question = "US American Team winning the Super Bowl";

float[] textEmbedding = await _aiVisionHelper.GetTextEmbedding(question);

Console.WriteLine($"Text embedding created...");

Text embedding created...


## Step 4: Store embeddings

The embeddings will be stored for further usage.

In [5]:
using System.Linq;

string textEmbeddingFile = "../../Assets/Embedding/MultiModalTextEmbedding.txt";

//Photorealistic image embedding
string imageEmbeddingFile = "../../Assets/Embedding/ImageEmbedding_Photorealistic.txt";
await File.WriteAllLinesAsync(
    imageEmbeddingFile, 
    imageEmbeddingPhotorealistic.ToArray().Select(vector => vector.ToString())
);

//Sketch image embedding
imageEmbeddingFile = "../../Assets/Embedding/ImageEmbedding_Sketch.txt";
await File.WriteAllLinesAsync(
    imageEmbeddingFile, 
    imageEmbeddingSketch.ToArray().Select(vector => vector.ToString())
);

//Oilpainting image embedding
imageEmbeddingFile = "../../Assets/Embedding/ImageEmbedding_Oilpainting.txt";
await File.WriteAllLinesAsync(
    imageEmbeddingFile, 
    imageEmbeddingOilpainting.ToArray().Select(vector => vector.ToString())
);

//Text embedding
await File.WriteAllLinesAsync(
    textEmbeddingFile, 
    textEmbedding.ToArray().Select(vector => vector.ToString())
);

Console.WriteLine($"Image embeddings stored...");
Console.WriteLine($"Text embedding stored...");


Image embeddings stored...
Text embedding stored...
